#LSTM stock price prediction

In [ ]:
#install required libraries 
!pip install yfinance
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19879 sha256=975073fb36e2c1ed948f956ebacfd4af74630b5ca36ebd858aead7716caed3ec
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [ ]:
# import libararies 
import math
import json
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, LSTM
import datetime as dt
from datetime import date
import getpass
from pyngrok import ngrok, conf
import os
import threading
from flask import Flask, jsonify
import requests
import subprocess

# list of stocks to predict
stocks = ['AAPL', 'AMZN', 'GOOG', 'MSFT', 'META', 'TSLA', 'DIS','JPM', 'KO', 'GME', 'NVDA', 'AMD', 'INTC', 'FORD']
today = date.today()

#Data Preprocessing

In [ ]:

# Create an empty dictionary to store prediction values and stock signals
predictions = {}
signals = {}

# loop for getting predition of each stock
for stock in stocks:
    # download stock data from Yahoo Finance API
    stock_data = yf.download(stock, start='2018-01-01', end=today)
    
    # extract closing prices
    closing_Arr = stock_data['Close']
    closing_Price = closing_Arr.values

    # Scale the data
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_prices = scaler.fit_transform(closing_Price.reshape(-1,1))

    # Create sequences of length 60
    seq_length = 60
    X = []
    y = []
    for i in range(seq_length, len(scaled_prices)):
        X.append(scaled_prices[i-seq_length:i, 0])
        y.append(scaled_prices[i, 0])

    X = np.array(X)
    y = np.array(y)

    # Reshape the input data for LSTM
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    
    # Define the LSTM model
    model = Sequential()
    model.add(LSTM(units=100, return_sequences=True, input_shape=(X.shape[1], 1)))
    model.add(LSTM(units=100))
    model.add(Dense(units=25))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X, y, epochs=1, batch_size=32)

    # Use the model to make predictions for the next 7 days
    next_days = []
    for i in range(7):
        last_seq = closing_Price.reshape(-1,1)[-seq_length:]
        last_seq_scaled = scaler.transform(last_seq)
        next_day_scaled = model.predict(np.array([last_seq_scaled]))
        next_day = scaler.inverse_transform(next_day_scaled)[0][0]
        next_days.append(next_day)
        closing_Price = np.append(closing_Price.reshape(-1,1), [[next_day]], axis=0)
        
    dates_stock = closing_Arr.index
    last_date = dates_stock[-1]

    # get the last date in the index
    last_date = closing_Arr.index[-1]

    # create an array of the next 8 days (-1) =7
    date_range = pd.date_range(last_date, periods=8, freq='B')
    
    #removing the first day as thats today`s closing price -1
    date_range = date_range[1:]

    # convert the array to a numpy array
    date_range_array = np.array(date_range)

    Dates = np.concatenate([dates_stock, date_range_array])
    # Convert the dates to strings in the format 'YYYY-MM-DD'
    Dates = Dates.astype('datetime64[D]').astype(str)

    # Add the predicted closing prices and dates to the dictionary
    predictions[stock] = {'Dates': Dates, 'close': closing_Price.tolist()} 
    print(stock)

    # NEW 
    #calculate the buy/sell signal
    stock_data = yf.download(stock, period='1y')
    
    # Calculate the 50-day and 200-day moving averages
    stock_data['MA50'] = stock_data['Close'].rolling(window=50).mean()
    stock_data['MA200'] = stock_data['Close'].rolling(window=200).mean()
    
    # Calculate the difference between the 50-day and 200-day moving averages
    stock_data['MA_diff'] = stock_data['MA50'] - stock_data['MA200']
    
    # Add a 'Signal' column indicating whether to buy or sell
    stock_data['Signal'] = 'Hold'
    stock_data.loc[stock_data['MA_diff'] > 0, 'Signal'] = 'Buy'
    stock_data.loc[stock_data['MA_diff'] < 0, 'Signal'] = 'Sell'
    
    # Print the latest signal
    latest_signal = stock_data.iloc[-1]['Signal']
    
    # Add the stock signal to the signals dictionary
    signals[stock] = latest_signal
    print(latest_signal)

[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 56ms/step
AAPL
[*********************100%***********************]  1 of 1 completed
Buy
[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 29ms/step
AMZN
[*********************100%***********************]  1 of 1 completed
Sell
[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 31ms/step
GOOG
[*********************100%***********************]  1 of 1 completed
Sell
[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 31ms/step
MSFT
[*********************100%***********************]  1 of 1 completed
Buy
[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 33ms/step
META
[*********************100%***********************]  1 of 1 completed
Buy
[**

#pyngrok

In [ ]:
#pyngrok to open a tunnel to that server.
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
authtoken= ""
conf.get_default().auth_token = authtoken

# Open a TCP ngrok tunnel to the SSH server
connection_string = ngrok.connect(22, "tcp").public_url

ssh_url, port = connection_string.strip("tcp://").split(":")
print(f" * ngrok tunnel available, access with `ssh root@{ssh_url} -p{port}`")

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth


 * ngrok tunnel available, access with `ssh root@2.tcp.ngrok.io -p19866`


#Flask

In [ ]:
#Flask app
os.environ["FLASK_ENV"] = "development"

app = Flask(__name__)
port = 5000

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port,subdomain="pricevision").public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL

@app.route("/")
def index():
    # Return a list of available stock symbols
    return "Disclaimer: The predictions provided are for informational purposes only and should not be considered as financial advice. We are not responsible for any financial investment losses. \n\nAvailable stocks: " + ", ".join(predictions.keys())

  # return api root url / stock symbol to get specific stock data
@app.route("/<string:symbol>")
def get_stock(symbol):
    if symbol in predictions:
        data = []
        dates = predictions[symbol]['Dates'][-8:]
        prices = predictions[symbol]['close'][-8:]
        for i in range(len(dates)):
            data.append({
                "date": dates[i],
                "price": prices[i][0]
            })
        # Convert the list of dictionaries to JSON format
        json_data = json.dumps(data, default=str, indent=4)
        print(json_data)

        # Create a Flask response with the JSON data and CORS headers
        response = jsonify(data)
        response.headers.add('Access-Control-Allow-Origin', '*')
       #return json_data
        return response

    else:
        return "Invalid stock symbol"

# Define the route to return the stock signals
@app.route("/<string:symbol>/signal")
def get_stock_signal(symbol):
    if symbol in signals:
        data = {
            "symbol": symbol,
            "signal": signals[symbol]
        }
        # Create a Flask response with the JSON data and CORS headers
        response = jsonify(data)
        response.headers.add('Access-Control-Allow-Origin', '*')
        return response
    else:
        return "Invalid stock symbol"

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()


 * ngrok tunnel "https://pricevision.ngrok.io" -> "http://127.0.0.1:5000"


'FLASK_ENV' is deprecated and will not be used in Flask 2.3. Use 'FLASK_DEBUG' instead.


 * Serving Flask app '__main__'


'FLASK_ENV' is deprecated and will not be used in Flask 2.3. Use 'FLASK_DEBUG' instead.
'FLASK_ENV' is deprecated and will not be used in Flask 2.3. Use 'FLASK_DEBUG' instead.


# disconnect and kill the ngrok and flask
ngrok.disconnect(public_url)
ngrok.kill()

#END